In [1]:
import pandas as pd
import numpy as np
import json
import heapq
import demoji
import re

In [2]:
preds = np.load('test_preds.npy').squeeze()
labels = np.load('test_labels.npy').squeeze()

In [18]:
fail_idx = np.where(preds != labels)[0]

In [7]:
df = pd.read_csv('./data/with_aug/test.csv')

In [20]:
df['match'] = True
df.loc[fail_idx, 'match'] = False

In [21]:
df.head()

,url,category,comment,label,match
0,https://www.bitchute.com/video/PbQ2FwGZFYOm/,news & politics,"""Minister Freeland"" = kyke controlling your ba...",yes,True
1,https://www.youtube.com/watch?v=EKgEj5asL3o,health,Americans: “MSG is unhealthy”\nAlso Americans:...,yes,True
2,https://www.youtube.com/watch?v=2tM1LFFxeKg,fitness,0:23 did I just hear Doomguy taking a missle t...,no,False
3,https://www.youtube.com/watch?v=oTRiwBmleGA,news & politics,Change the word begging instead humiliation,yes,True
4,https://www.youtube.com/watch?v=WKa11G1Jpfs,religion,Gaitonde was right.. 21 din hai aapke pas. Bac...,no,True


In [18]:
df[df['match'] == False].to_csv('test.csv')

In [2]:
comments = pd.read_csv('./data/with_aug/train.csv')
metadata = pd.read_csv('./data/extra_data_trans.csv')

In [3]:
df = pd.merge(comments, metadata, how='left', on='url')

In [17]:
df[df.isnull().any(axis=1)][['url', 'transcript']].drop_duplicates(subset=['url', 'transcript'])

,url,transcript
202,https://www.youtube.com/watch?v=glR4fKs7WSU,"But good morning, although this is. Certainly...."
329,https://www.bitchute.com/video/GLWib2Ae8PCx/,So some of you just don't believe that my dog'...
332,https://www.youtube.com/watch?v=lWQONFEHqpc,NaN
340,https://www.youtube.com/watch?v=0BTlDRddyK4,Yeah.
500,https://www.youtube.com/watch?v=3gcQrFsUFzQ,NaN
503,https://www.bitchute.com/video/ZruneWL9UZId/,NaN
504,https://www.youtube.com/watch?v=yDAcEJmqY6U,NaN
522,https://www.youtube.com/watch?v=1SVLhgWHm1A,"Hayden getting really excited, though. I think..."
694,https://www.youtube.com/watch?v=ds5MbgbXLBo,Home cooks tonight. You will be replicating a ...
775,https://www.youtube.com/watch?v=Hyfc_MLpkRw,NaN


In [1]:
import json

In [12]:
with open('./non_hate_youtube_data.json') as f:
    cur_data = json.load(f)

In [13]:
with open('./metadata/non_hate_youtube_data.json') as f:
    prev_data = json.load(f)

In [14]:
for video_data in prev_data:
    if video_data['url'] in ['https://www.youtube.com/watch?v=O_AHDyUmqs', 'https://www.youtube.com/watch?v=G>gw6erIp_Qc']:
        print("hi")
        cur_data.append(video_data)

In [11]:
with open('./youtube_data.json', 'w') as f:
    json.dump(cur_data, f)

## Extracting key phrases beforehand

In [4]:
from keybert import KeyBERT

In [2]:
metadata = pd.read_csv('./data/extra_data_trans.csv')

In [45]:
metadata.head()

,url,title,desc,transcript,key_phrases_desc_bert,key_phrases_transcript_bert,key_phrases_desc_long,key_phrases_transcript_long
0,https://www.youtube.com/watch?v=C0MjY_tpG-I,NZ Māori Party calls for a ‘divorce’ from the ...,Sky News host Peta Credlin says the New Zealan...,It was yesterday we were celebrating the recor...,monarchy ms peta party calling divorce credlin...,maori party ideas haven cancelling queen quest...,monarchy ms peta party calling divorce credlin...,maori party ideas haven cancelling queen quest...
1,https://www.youtube.com/watch?v=vXoqfoAXR44,Is India preparing to take back PoK? Air Force...,"Pakistan should start counting its days, as Pr...",Prime Minister Narendra Modi has finally put h...,pakistan start counting pedal illegally occupi...,soon capture kashmir gas pedal illegally china...,pakistan start counting pedal illegally occupi...,soon capture kashmir gas pedal illegally china...
2,https://www.youtube.com/watch?v=mndBZFdPyVw&t=...,How To Make Veg Enchiladas | Vegetarian Enchil...,Veg Enchiladas Recipe | How To Make Enchiladas...,"Hi guys, welcome to Rajshri Food. This is the ...",veg enchiladas recipe rajshri filling tbsp sur...,making bean enchiladas salt absolutely check b...,veg enchiladas recipe rajshri filling tbsp sur...,making bean enchiladas salt absolutely check b...
3,https://www.youtube.com/watch?v=6a_ONOdfJTU,CNN’s Medical Expert Gives HORRIFYING Reason W...,Enjoy Healthy Aging Support of Collagen \nhttp...,We're witnessing a major narrative shift when ...,collagen http healthwithgary mask mandates pre...,mandates end cnn yeah mean worried decrease ho...,collagen http healthwithgary mask mandates pre...,mandates end cnn yeah mean worried decrease ho...
4,https://www.youtube.com/watch?v=27q2nnvwfUQ&t=14s,Salma Hayek Reveals What Her Daughter Valentin...,“Extra” Special Correspondent Samantha Harris ...,Hello Hello beauty you got looking fabulous an...,watch eternals daughter valentina salma harris...,young latina women talking superhero calling h...,watch eternals daughter valentina salma harris...,young latina women talking superhero calling h...


In [34]:
kw_model = KeyBERT()

In [5]:
def preprocess(text=''):
    if text != text:
        return ''
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def process_keyphrase_text(kw_model, text, max_length, key_phrase_count):
    doc = ' '.join(text.split()[:max_length])
    keywords = kw_model.extract_keywords(doc, top_n=key_phrase_count, use_mmr=True,
                                            diversity=0.7, keyphrase_ngram_range=[1, 3])
    doc = ' '.join([keyword[0] for keyword in keywords])
    return doc

In [ ]:
print(preprocess('SKy news'))
process_keyphrase_text(kw_model, 'SKy news', 200, 50)


In [ ]:
metadata['key_phrases_desc_bert'] = metadata['desc'].apply(lambda x: process_keyphrase_text(kw_model, preprocess(x), 200, 50))

In [ ]:
metadata['key_phrases_transcript_bert'] = metadata['transcript'].apply(lambda x: process_keyphrase_text(kw_model, preprocess(x), 1000, 150))

In [ ]:
metadata['key_phrases_desc_long'] = metadata['desc'].apply(lambda x: process_keyphrase_text(kw_model, preprocess(x), 300, 100))

In [ ]:
metadata['key_phrases_transcript_long'] = metadata['transcript'].apply(lambda x: process_keyphrase_text(kw_model, preprocess(x), 2400, 400))

In [46]:
metadata.to_csv('./data/extra_data_trans.csv')

## Compute other comment metadata

In [12]:
# TODO
# filter top 1000 comments for all videos. Store the comment and url.
# process the text, remove the links, mentions and add the emojis back
# get the similarity score between the dataset comments & the other comments
# Renormalize the score and filter to take the middle [0.2-0.8] comments. 
# Find the key phrases for these comments and store it 

In [62]:
with open('metadata/youtube_data.json') as f:
    ytHateData = json.load(f)

In [63]:
with open('metadata/non_hate_youtube_data.json') as f:
    ytNonHateData = json.load(f)

In [64]:
with open('metadata/bitchute_data.json') as f:
    bitchuteData = json.load(f)

In [65]:
comments = {}

In [66]:
for data in ytHateData:
    comments[data['url']] = []
    heap = []
    for comment in data['comments']:
        if comment['top_comment']['totalReplyCount'] > 0:
            comments[data['url']].append(comment['top_comment']['textOriginal'])
        if len(heap) > 1000:
            if heap[0][0] < comment['top_comment']['likeCount']:
                heapq.heappushpop(heap, [comment['top_comment']['likeCount'], comment['top_comment']['textOriginal']])
        else:
            heapq.heappush(heap, [comment['top_comment']['likeCount'], comment['top_comment']['textOriginal']])
    n = 1000 - len(comments[data['url']])
    top_comments = heapq.nlargest(n, heap, key=lambda x: x[0])
    for comment in top_comments:
        comments[data['url']].append(comment[1])

In [67]:
for data in ytNonHateData:
    if data['url'] not in comments:
        comments[data['url']] = []
    else:
        continue
    heap = []
    for comment in data['comments']:
        if comment['top_comment']['totalReplyCount'] > 0:
            comments[data['url']].append(comment['top_comment']['textOriginal'])
        if len(heap) > 1000:
            if heap[0][0] < comment['top_comment']['likeCount']:
                heapq.heappushpop(heap, [comment['top_comment']['likeCount'], comment['top_comment']['textOriginal']])
        else:
            heapq.heappush(heap, [comment['top_comment']['likeCount'], comment['top_comment']['textOriginal']])
    n = 1000 - len(comments[data['url']])
    top_comments = heapq.nlargest(n, heap, key=lambda x: x[0])
    for comment in top_comments:
        comments[data['url']].append(comment[1])

In [68]:
for data in bitchuteData:
    comments[data['url']] = []
    heap = []
    for comment in data['comments']['values']:
        if len(heap) > 1000:
            if heap[0][0] < comment[10]:
                heapq.heappushpop(heap, [comment[10], comment[4]])
        else:
            heapq.heappush(heap, [comment[10], comment[4]])
    n = 1000 - len(comments[data['url']])
    top_comments = heapq.nlargest(n, heap, key=lambda x: x[0])
    for comment in top_comments:
        comments[data['url']].append(comment[1])

In [69]:
comments['https://www.bitchute.com/video/DFlrLnZgwjaM/']

['Show this to any BLM and they will make excuses and justify his actions.😂brain washed niggers',
 "He's not JUST racist. They are satanists, stealing and eating our children. \n \n(EVIDENCE LINKS BELOW) PRAY!!!! #GODisFORus!! Our Corrupt Leaders are under judgement and are being removed. Ask Jesus what to do, and you will outsmart our enemies, just as President Trump has been doing for 4 years. Trump will have a second term. SEE the rally sizes via youtube videos; Trump has thousands and thousands of supporters at each and every rally, while Biden is hard pressed to gain a hundred people. In fact, his prior rallies were 19 people, and 12 people before that were half media. \n \nEVIDENCE OF CORRUPT DEMOCRAPS -- Sidney Powell-Weiner laptop-HRC server • 98,603 views -- https://www.youtube.com/watch?v=s2JZffZUGhg -- Mar 12, 2020\n \nDEMS DAMNING SATANIC EVIDENCE - WARNING: ACTUAL CRIME SCENE PHOTOS ON THIS PAGE -- DO NOT VIEW IN PRESENCE OF CHILDREN -- https://beforeitsnews.com/alternativ

In [70]:
with open('top_other_comments_all_data.json', 'w') as f:
    json.dump(comments, f, indent=2)

### Text processing, get score with comments in dataset

In [2]:
with open('top_other_comments_all_data.json', 'r') as f:
    comments = json.load(f)

In [9]:
def process_text(text=''):
    if text != text:
        return ''
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        t = re.sub(r'http\S+', '', t)
        new_text.append(t)
    new_text = " ".join(new_text)
    new_text = demoji.replace_with_desc(new_text, sep=' ')
    return new_text

In [4]:
demoji.replace_with_desc("🤣Real People don't need QUEENS or Kings.", sep=' ')

" rolling on the floor laughing Real People don't need QUEENS or Kings."

In [5]:
for url, comment_arr in comments.items():
    for i in range(len(comment_arr)):
        comment_arr[i] = process_text(comment_arr[i])

KeyboardInterrupt: 

In [75]:
for url, comment_arr in comments.items():
    for i in range(len(comment_arr)):
        comment_arr[i] = re.sub('\\s+', ' ', comment_arr[i])

In [76]:
with open('top_other_comments_all_data.json', 'w') as f:
    json.dump(comments, f, indent=2)

In [77]:
s = "Did she just say that a Republic, the USA, isn\\u2019t free? It was so intelligent till right at the end."
s.encode('ascii').decode('unicode-escape').encode('utf-16', 'surrogatepass').decode('utf-16')

'Did she just say that a Republic, the USA, isn’t free? It was so intelligent till right at the end.'

In [78]:
comments['https://www.youtube.com/watch?v=C0MjY_tpG-I'][2]

'The queen of England hasn’t been paying her rent to the Maori people since 1840 she owns the Maori people $8 trillion dollars Plus GST and interest'

In [79]:
lens = []
sum = 0
for url, arr in comments.items():
    lens.append(len(arr))
    sum+=len(arr)

In [80]:
len(lens)

431

## Handle bleurt scores

In [7]:
with open('bleurt_results.json', 'r') as f:
    data = json.load(f)

In [8]:
for i, comment_data in enumerate(data):
    comment_data['scores'] = sorted(comment_data['scores'], key=lambda x: x[1], reverse=True)

In [9]:
num_comments_per_url = {}
for i, comment_data in enumerate(data):
    if comment_data['url'] not in num_comments_per_url:
        num_comments_per_url[comment_data['url']] = 1
    else:
       num_comments_per_url[comment_data['url']] += 1 

In [10]:
import numpy as np
  
# explicit function to normalize array
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr
  
range_to_normalize = (0, 1)


In [15]:

for i, comment_data in enumerate(data):
    arr = []
    for comment, score in comment_data['scores']:
        arr.append(score)

    normalized_array_1d = normalize(
        arr, range_to_normalize[0], 
    range_to_normalize[1])
    for i, comment_score_pair in enumerate(comment_data['scores']):
        comment_data['scores'][i][1] = normalized_array_1d[i]


In [45]:
data[15]['comment'], data[15]['scores'][300:350]

('Me: sneeze* Periods: ORAORAORAORA',
 [['A period is pretty much just our uterus punishing us for not getting pregnant. *"You had ONE job!"*',
   0.1543544977903366],
  ["I remember telling my boyfriend I was on my period, he asked me how I know I'm on my period every month. unamused face Edit: He's an ex now",
   0.1539486050605774],
  ['Bruh i have my period rrom The mormintele and that jeans that im pregnant? face with tears of joy I’m just 9 years old I will be 10 in 2 months',
   0.15297424793243408],
  ['People with periods not just girls just saying', 0.1523372083902359],
  ["I'm 11, and I think I'm on my period.. I was shocked when I found out",
   0.15216313302516937],
  ['Why do humans have periods more often than other mammals?',
   0.15187081694602966],
  ['Yep. No babies for me!', 0.15180999040603638],
  ["Wait- My dog bleeds everynow and then down there. So you're telling me my female dog doesn't have a period?",
   0.1518002450466156],
  ['I hate my life I wanna be a ma

In [46]:
data_to_store = []
for i, comment_data in enumerate(data):
    one_data_to_store = {}
    one_data_to_store['url'] = comment_data['url']
    one_data_to_store['comment'] = comment_data['comment']
    one_data_to_store['other_comments'] = set()
    for comment, score in comment_data['scores'][num_comments_per_url[comment_data['url']]:]:
        if score > 0.15:
            one_data_to_store['other_comments'].add(comment)
    one_data_to_store['other_comments'] = list(one_data_to_store['other_comments'])
    data_to_store.append(one_data_to_store)

In [47]:
with open('other_comments_metadata.json', 'w') as f:
    json.dump(data_to_store, f, indent=2)

In [1]:
from keybert import KeyBERT
import json

In [2]:
with open('other_comments_metadata.json', 'r') as f:
    metadata_other_comments = json.load(f)

In [3]:
kw_model = KeyBERT()

In [13]:
for i, comment_data in enumerate(metadata_other_comments):
    metadata_other_comments[i]['combined_text'] = ' '.join(comment_data['other_comments'])
    del metadata_other_comments[i]['other_comments']
    comment_data['key_phrases_other_comments'] = process_keyphrase_text(kw_model, preprocess(text), 2400, 400)

Last clean up

In [85]:
with open('keyphrases_other_comments.json', 'r') as f:
    data = json.load(f)

In [86]:
for i, comment_data in enumerate(data):
    del data[i]['combined_text']

In [87]:
with open('keyphrases_other_comments.json', 'w') as f:
    json.dump(data, f, indent=2)

In [88]:
csv_to_store = {"url": [], "comment": [], "key_phrases_other_comments": []}
for i, comment_data in enumerate(data):
    csv_to_store['url'].append(comment_data['url'])
    csv_to_store['comment'].append(comment_data['comment'])
    csv_to_store['key_phrases_other_comments'].append(comment_data['key_phrases_other_comments'])

In [89]:
import pandas as pd

df = pd.DataFrame(csv_to_store)

In [90]:
df.head()

,url,comment,key_phrases_other_comments
0,https://www.youtube.com/watch?v=BhSucJ1KDcU,Un sorry I can’t unsee this. mindless pretenders.,instead kim beautiful covert promotions fail r...
1,https://www.youtube.com/watch?v=BhSucJ1KDcU,i'm sorry but i can't see this,vogue empowerment kim realistic inclusivity sa...
2,https://www.youtube.com/watch?v=BhSucJ1KDcU,i'm sorry i can't see,beauty standards celebrities great video sayin...
3,https://www.youtube.com/watch?v=BhSucJ1KDcU,i'm sorry i can't see it,kim video body generations think ironic heart ...
4,https://www.youtube.com/watch?v=BhSucJ1KDcU,i'm sorry i can't see this,kim video body especially younger generations ...


In [91]:
df.to_csv('data/extra_data_other_comments.csv', index=False)

## Checking which comments are being incorrectly classified

In [2]:
labels = np.load('./npy_files/test_labels_ethereal-blaze-26.npy')
preds = np.load('./npy_files/test_preds_ethereal-blaze-26.npy')

In [3]:
comments = pd.read_csv('data/with_aug/test.csv')

In [4]:
fail_idx = np.where(np.not_equal(preds, labels))[0]

In [5]:
failed_preds = comments.loc[fail_idx]

In [6]:
failed_preds.to_csv('model_failed_results.csv')

## Checking training data for nan

In [3]:
metadata = pd.read_csv('data/extra_data_other_comments.csv')

In [4]:
metadata = metadata.replace(np.nan, '', regex=True)

In [17]:
metadata[metadata.isnull().any(axis=1)]

,url,comment,key_phrases_other_comments


In [5]:
metadata.to_csv('data/extra_data_other_comments.csv', index=False)

### Cleaning with_aug data

In [6]:
train_comment = pd.read_csv('data/hate_towards_whom_aug/train.csv')
test_comment = pd.read_csv('data/hate_towards_whom_aug/test.csv')

In [7]:
train_comment.target.value_counts().sum()

6485

In [10]:
for i, row in train_comment.iterrows():
    train_comment.iloc[i]['comment'] = process_text(train_comment.iloc[i]['comment'])
    train_comment.iloc[i]['comment'] = re.sub('\\s+', ' ', train_comment.iloc[i]['comment'])

for i, row in test_comment.iterrows():
    test_comment.iloc[i]['comment'] = process_text(test_comment.iloc[i]['comment'])
    test_comment.iloc[i]['comment'] = re.sub('\\s+', ' ', test_comment.iloc[i]['comment'])

In [11]:
train_comment.drop_duplicates(inplace=True)
test_comment.drop_duplicates(inplace=True)

In [12]:
train_comment.to_csv('data/hate_towards_whom_aug/train.csv', index=False)
test_comment.to_csv('data/hate_towards_whom_aug/test.csv', index=False)

### Merging other comments checks

In [13]:
train_merged = pd.merge(train_comment, metadata, how='left', on=['url', 'comment'])
test_merged = pd.merge(test_comment, metadata, how='left', on=['url', 'comment'])

In [14]:
trans_merged = pd.read_csv('data/extra_data_trans.csv')

In [15]:
trans_merged.head(3)

,url,title,desc,transcript,key_phrases_desc_bert,key_phrases_transcript_bert,key_phrases_desc_long,key_phrases_transcript_long
0,https://www.youtube.com/watch?v=C0MjY_tpG-I,NZ Māori Party calls for a ‘divorce’ from the ...,Sky News host Peta Credlin says the New Zealan...,It was yesterday we were celebrating the recor...,monarchy ms peta party calling divorce credlin...,maori party ideas haven cancelling queen quest...,monarchy ms peta party calling divorce credlin...,maori party ideas haven cancelling queen quest...
1,https://www.youtube.com/watch?v=vXoqfoAXR44,Is India preparing to take back PoK? Air Force...,"Pakistan should start counting its days, as Pr...",Prime Minister Narendra Modi has finally put h...,pakistan start counting pedal illegally occupi...,soon capture kashmir gas pedal illegally china...,pakistan start counting pedal illegally occupi...,soon capture kashmir gas pedal illegally china...
2,https://www.youtube.com/watch?v=mndBZFdPyVw&t=...,How To Make Veg Enchiladas | Vegetarian Enchil...,Veg Enchiladas Recipe | How To Make Enchiladas...,"Hi guys, welcome to Rajshri Food. This is the ...",veg enchiladas recipe rajshri filling tbsp sur...,making bean enchiladas salt absolutely check b...,veg enchiladas recipe rajshri filling tbsp sur...,making bean enchiladas salt absolutely check b...


In [24]:
test_df = pd.merge(test_merged, trans_merged, how='left', on='url')
train_df = pd.merge(train_merged, trans_merged, how='left', on='url')

In [17]:
df.columns

Index(['url', 'category', 'comment', 'label', 'target',
       'key_phrases_other_comments', 'title', 'desc', 'transcript',
       'key_phrases_desc_bert', 'key_phrases_transcript_bert',
       'key_phrases_desc_long', 'key_phrases_transcript_long'],
      dtype='object')

In [18]:
new_text='WHITE PEOPLE 🙎🏻‍♀️🙎🏻‍♂️ IN THEIR FINEST MOMENT 👍🏻👍🏼👍🏽'
demoji.replace_with_desc(new_text, sep=' ')

'WHITE PEOPLE  woman pouting: light skin tone  man pouting: light skin tone  IN THEIR FINEST MOMENT  thumbs up: light skin tone  thumbs up: medium-light skin tone  thumbs up: medium skin tone '

In [26]:
train_df.to_csv('train_combined.csv')

In [27]:
test_df.to_csv('test_combined.csv')

In [41]:
train_df[train_df['comment'].str.contains('"Ew, I don\'t eat msg seasoned food" said Karen while smoking her cigarette')].to_csv('test_example.csv')

In [26]:
df['comment'][0], df['title'][0], df['key_phrases_desc_bert'][0], df['key_phrases_transcript_bert'][0], df['key_phrases_other_comments'][0], df['label'][0]

('"Minister Freeland" = kyke controlling your bank account.',
 'Canada tells people how they can get their bank accounts unfrozen if they donated to the Trucker c..',
 'trucker convoy liberal shutdown anti vaccine bank accounts canada tells people movements look reacted agree unfrozen donated machine machine does antifa versus mandate protests http ones raging blm react does agree support liberal think look people versus donated trucker mandate does tells reacted tells people bank canada reacted blm antifa anti vaccine mandate protests shutdown raging machine machine protests http think ones accounts unfrozen look reacted blm support movements react anti shutdown liberal versus react anti support donated trucker convoy trucker donated agree support movements convoy mandate protests vaccine does agree convoy liberal think liberal think ones',
 'banks emergencies act constituents donated according rcmp rcmp unfrozen stop blockade occupation person certain account frozen mistaken use powe

In [105]:
train_merged[train_merged.isnull().any(axis=1)]

,url,category,comment,label,key_phrases_other_comments


In [106]:
test_merged[test_merged.isnull().any(axis=1)]

,url,category,comment,label,key_phrases_other_comments


In [107]:
print(len(test_merged))
print(len(train_merged))

208
6530


1. comment
2. comment + title
3. comment + description
4. comment + transcript
5. comment + other_comments
6. comment + title + description
7. comment + title + description + transcript
8. comment + title + description + transcript + other_comments

### Testing some stats

In [8]:
train_comment = pd.read_csv('data/with_aug/train.csv')
test_comment = pd.read_csv('data/with_aug/test.csv')

In [9]:
len(train_comment)

6529

In [11]:
train_comment.drop_duplicates()

,url,category,comment,label
0,https://www.youtube.com/watch?v=BhSucJ1KDcU,fashion & beauty,Un sorry I can’t unsee this. mindless pretenders.,yes
1,https://www.youtube.com/watch?v=BhSucJ1KDcU,fashion & beauty,i'm sorry but i can't see this,yes
2,https://www.youtube.com/watch?v=BhSucJ1KDcU,fashion & beauty,i'm sorry i can't see,yes
3,https://www.youtube.com/watch?v=BhSucJ1KDcU,fashion & beauty,i'm sorry i can't see it,yes
4,https://www.youtube.com/watch?v=BhSucJ1KDcU,fashion & beauty,i'm sorry i can't see this,yes
...,...,...,...,...
6524,https://www.youtube.com/watch?v=pG7mHUnW06Q,news & politics,the lebonese women are beautiful,no
6525,https://www.youtube.com/watch?v=pG7mHUnW06Q,news & politics,those lebonese women are stunning,no
6526,https://www.youtube.com/watch?v=pG7mHUnW06Q,news & politics,lebonese women are gorgeous,no
6527,https://www.youtube.com/watch?v=pG7mHUnW06Q,news & politics,the lebonese women are gorgeous,no


In [5]:
len(test_comment)

208

Making the data single target

In [2]:
train_df = pd.read_csv('./data/hate_towards_whom_aug/train.csv')
test_df = pd.read_csv('./data/hate_towards_whom_aug/test.csv')

In [5]:
train_df['target'].value_counts()

None                       2927
Individual                 1650
Community                  1305
Location                    251
Organisation                177
Community,Individual         84
Community,Organisation       39
Community,Location           24
Individual,Organisation      22
Location,Organisation         6
Name: target, dtype: int64

In [8]:
for i, row in train_df.iterrows():
    targets = row['target'].split(',')
    if len(targets) == 1:
        continue
    if targets[0] == 'Community':
        if targets[1] == 'Individual':
            train_df['target'][i] = targets[0]
        else:
           train_df['target'][i] = targets[1]
    elif targets[0] == 'Individual':
        train_df['target'][i] = 'Organisation'
    elif targets[0] == 'Location':
        train_df['target'][i] = targets[0]

In [9]:
train_df['target'].value_counts()

None            2927
Individual      1650
Community       1389
Location         281
Organisation     238
Name: target, dtype: int64

In [10]:
for i, row in test_df.iterrows():
    targets = row['target'].split(',')
    if len(targets) == 1:
        continue
    if targets[0] == 'Community':
        if targets[1] == 'Individual':
            test_df['target'][i] = targets[0]
        else:
           test_df['target'][i] = targets[1]
    elif targets[0] == 'Individual':
        test_df['target'][i] = 'Organisation'
    elif targets[0] == 'Location':
        test_df['target'][i] = targets[0]

In [11]:
test_df['target'].value_counts()

None            102
Individual       50
Community        32
Location         12
Organisation     11
Name: target, dtype: int64

In [12]:
test_df.to_csv('./data/hate_towards_whom_aug_single/test.csv', index=False)
train_df.to_csv('./data/hate_towards_whom_aug_single/train.csv', index=False)